In [2]:
def power(base:float, power:float=2) -> float:
    return base ** power

In [5]:
power(5)

25

In [6]:
power(5,3)

125

- http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/
- https://en.wikipedia.org/wiki/Association_rule_learning

In [113]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [112]:
dfRaw = pd.read_csv('data/market_basket.csv', header=None, delimiter=';', names=['transaction'])

def transaction_process(transaction:list) -> list:
    transaction = list(filter(lambda product: product != '', transaction))
    transaction = list(map(lambda prod: prod.strip().replace(' ', '_'), transaction))
    return transaction

transactions = dfRaw.transaction.str.split(',')
transactions = transactions.map(transaction_process)

encoder = TransactionEncoder()
data = encoder.fit_transform(transactions)
columns = encoder.columns_

dfTrans = pd.DataFrame(data, columns=columns)
dfTrans.head()

,almonds,antioxydant_juice,asparagus,avocado,babies_food,bacon,barbecue_sauce,black_tea,blueberries,body_spray,...,turkey,vegetables_mix,water_spray,white_wine,whole_weat_flour,whole_wheat_pasta,whole_wheat_rice,yams,yogurt_cake,zucchini
0,True,True,False,True,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [120]:
dfFreq = apriori(dfTrans, min_support=0.01, use_colnames=True)
dfFreq

,support,itemsets
0,0.020397,(almonds)
1,0.033329,(avocado)
2,0.010799,(barbecue_sauce)
3,0.014265,(black_tea)
4,0.011465,(body_spray)
...,...,...
252,0.011065,"(mineral_water, ground_beef, milk)"
253,0.017064,"(spaghetti, mineral_water, ground_beef)"
254,0.015731,"(spaghetti, mineral_water, milk)"
255,0.010265,"(spaghetti, olive_oil, mineral_water)"


In [123]:
dfRule = association_rules(dfFreq, metric='confidence', min_threshold=0.01)
dfRule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(mineral_water),(avocado),0.238368,0.033329,0.011598,0.048658,1.459926,0.003654,1.016113
1,(avocado),(mineral_water),0.033329,0.238368,0.011598,0.348000,1.459926,0.003654,1.168147
2,(burgers),(cake),0.087188,0.081056,0.011465,0.131498,1.622319,0.004398,1.058080
3,(cake),(burgers),0.081056,0.087188,0.011465,0.141447,1.622319,0.004398,1.063198
4,(burgers),(chocolate),0.087188,0.163845,0.017064,0.195719,1.194537,0.002779,1.039630
...,...,...,...,...,...,...,...,...,...
427,"(spaghetti, mineral_water)",(pancakes),0.059725,0.095054,0.011465,0.191964,2.019529,0.005788,1.119933
428,"(pancakes, mineral_water)",(spaghetti),0.033729,0.174110,0.011465,0.339921,1.952333,0.005593,1.251198
429,(spaghetti),"(pancakes, mineral_water)",0.174110,0.033729,0.011465,0.065850,1.952333,0.005593,1.034385
430,(pancakes),"(spaghetti, mineral_water)",0.095054,0.059725,0.011465,0.120617,2.019529,0.005788,1.069244


## Pipeline:

1. Rời rạc hoá các trường continuous về dạng categorical. Phương pháp:
   - quantiles (đơn giản, dễ dùng)
   - kmeans (đơn giản, lợi thế hơn quantiles là có thể dùng cho tổ hợp nhiều trường continuous)
   - decision tree (sử dụng target để encoding có thể dẫn đến data leakage)
2. Clean một số trường categorical. VD: tách tháng từ NTZK_RGSTDDATE, tách giờ từ NTZK_RGSTDTIME, tách mã HS cha.
3. Tổng hợp domain knowledge dưới dạng rule "if-else". VD: tờ khai với mã HS NTZR_HSCODE = XXXXXXXX, NTZK_RGSTDHOUR là buổi tối, unloading port = VNXFH chắc chắn vi phạm. Rule xác định tờ khai chắc chắn không vi phạm cũng rất cần thiết. Các rule này được dùng để correct lại nhãn.
4. One-hot encode tất cả các trường categorical. Implement bằng hàm `get_dummies` của pandas.
5. Sử dụng thư viện mlxtend: `apriori` để tính support cho các tổ hợp phổ biến và `association_rules` để tìm ra các consequent có giá trị TARGET=0 mà có confidence/lift cao.
6. Filter bớt các bản ghi nằm trong rule TARGET=0. Việc này k thay đổi rule xác định TARGET=1 nhưng sẽ làm tăng confidence của các rule đó. Ngoài ra sẽ làm dataset nhẹ hơn và giảm tình trạng mất cân bằng. Nếu train mô hình ML lên dataset này sẽ cho kết quả tốt hơn dataset mất cân bằng ban đầu.
7. Dùng association rules tìm ra các rule với TARGET=1.

In [1]:
import pandas as pd
import numpy as np

In [10]:
columns = [
    'CRT_KANSYO',
    'NTZK_TRNSPTMTHDFORSRCHCODE',
    'NTZK_DCLRTNKINDCODE',
    'NTZK_RGSTDDATE',
    'NTZK_RGSTDTIME',
    'NTZK_CNTRYCODE',
    'NTZK_UNLOADINGPORTCODE',
    'NTZK_LOADINGLOCTNCODE',
    'NTZK_PYMTTERM',
    'NTZK_TTLTAXVAL',
    'NTZK_TTLPAYABLETAX',
    'NTZR_HSCODE',
    'NTZR_QTY_1',
    'NTZR_QTYUNITCODE_1',
    'NTZR_IMPTAXRATE',
    'NTZR_IMPTAXAMNT',
    'NTZR_ORIGINPLCCODE',
    'INPUT'
]

# df = pd.read_csv('data/Dữ liệu test.txt', nrows=100000, delimiter='\t', usecols=columns,
#                  dtype={'NTZR_HSCODE': str, 'NTZK_RGSTDDATE': str, 'NTZK_RGSTDTIME': str})
# df.to_csv('data/apriori.csv', index=False)

df = pd.read_csv('data/apriori.csv', dtype={'NTZR_HSCODE': str, 'NTZK_RGSTDDATE': str, 'NTZK_RGSTDTIME': str})

df1 = df.query('INPUT==0').sample(frac=0.1, random_state=7)
df2 = df.query('INPUT==1')
df = df1.append(df2)
df = df.astype({'INPUT': str})
df.head()

,CRT_KANSYO,NTZK_TRNSPTMTHDFORSRCHCODE,NTZK_DCLRTNKINDCODE,NTZK_RGSTDDATE,NTZK_RGSTDTIME,NTZK_CNTRYCODE,NTZK_UNLOADINGPORTCODE,NTZK_LOADINGLOCTNCODE,NTZK_PYMTTERM,NTZK_TTLTAXVAL,NTZK_TTLPAYABLETAX,NTZR_HSCODE,NTZR_QTY_1,NTZR_QTYUNITCODE_1,NTZR_IMPTAXRATE,NTZR_IMPTAXAMNT,NTZR_ORIGINPLCCODE,INPUT
35898,03NK,O,A12,20200104,143346,VN,VNXFH,VNZZZ,KC,3.155070e+07,NaN,48191000,30.0,PCE,NaN,NaN,VN,0
40375,03NK,O,E11,20200109,142008,VN,VNXFH,VNZZZ,KC,2.822824e+08,NaN,39269099,14400.0,PCE,NaN,NaN,VN,0
53403,47XE,O,E15,20200106,145718,VN,VNZZZ,VNZZZ,TTR,2.865361e+08,NaN,72119099,250.8,KGM,NaN,NaN,VN,0
11361,01PR,O,A12,20200106,102305,VN,VNZZZ,VNZZZ,KC,6.048526e+08,NaN,76169990,5.0,PCE,NaN,NaN,VN,0
70181,01PR,O,E15,20200107,084934,VN,VNXRL,VNZZZ,KC,1.451513e+09,NaN,48089090,82.0,PCE,NaN,NaN,VN,0


In [11]:
dfProcessed = df.copy()

dfProcessed['NTZR_HSCODE_2'] = dfProcessed.NTZR_HSCODE.str[0:2]
# dfProcessed['NTZR_HSCODE_4'] = dfProcessed.NTZR_HSCODE.str[0:4]
# dfProcessed['NTZR_HSCODE_6'] = dfProcessed.NTZR_HSCODE.str[0:6]
# dfProcessed['NTZR_HSCODE_8'] = dfProcessed.NTZR_HSCODE.str[0:8]

# dfProcessed['NTZK_RGSTDMONTH'] = dfProcessed.NTZK_RGSTDDATE.str[4:6]
dfProcessed['NTZK_RGSTDHOUR'] = dfProcessed.NTZK_RGSTDTIME.str[0:2]

labels = dfProcessed.NTZK_TTLTAXVAL.quantile(np.linspace(0,1,11)).values.astype(int)[1:]
dfProcessed['NTZK_TTLTAXVAL'] = pd.qcut(dfProcessed.NTZK_TTLTAXVAL, q=10, labels=labels)

columns = [
    'CRT_KANSYO',
    'NTZK_TRNSPTMTHDFORSRCHCODE',
    'NTZK_DCLRTNKINDCODE',
    'NTZK_UNLOADINGPORTCODE',
    'NTZK_PYMTTERM',
    'NTZK_TTLTAXVAL',
    'NTZR_HSCODE_2',
    'NTZK_RGSTDHOUR',
    'INPUT',
]
dfProcessed = dfProcessed[columns]
dfProcessed = pd.get_dummies(dfProcessed)

dfProcessed.columns.tolist()
dfProcessed = dfProcessed.astype(bool)
dfProcessed.head()

,CRT_KANSYO_01IK,CRT_KANSYO_01M1,CRT_KANSYO_01NV,CRT_KANSYO_01PJ,CRT_KANSYO_01PQ,CRT_KANSYO_01PR,CRT_KANSYO_02F1,CRT_KANSYO_02F2,CRT_KANSYO_02F3,CRT_KANSYO_02PG,...,NTZK_RGSTDHOUR_16,NTZK_RGSTDHOUR_17,NTZK_RGSTDHOUR_18,NTZK_RGSTDHOUR_19,NTZK_RGSTDHOUR_20,NTZK_RGSTDHOUR_21,NTZK_RGSTDHOUR_22,NTZK_RGSTDHOUR_23,INPUT_0,INPUT_1
35898,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
40375,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
53403,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
11361,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
70181,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [25]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
dfFreq = apriori(dfProcessed, use_colnames=True, min_support=0.2)
dfFreq.head()

,support,itemsets
0,0.995121,(NTZK_TRNSPTMTHDFORSRCHCODE_O)
1,0.542069,(NTZK_DCLRTNKINDCODE_E15)
2,0.503137,(NTZK_UNLOADINGPORTCODE_VNZZZ)
3,0.758040,(NTZK_PYMTTERM_KC)
4,0.231803,(NTZR_HSCODE_2_39)


In [28]:
dfRules = dfFreq.copy()
# dfRules = dfRules[dfRules.itemsets.astype(str).str.contains('INPUT_0')]
dfRules = association_rules(dfRules, metric='confidence')
dfRules = dfRules[dfRules.consequents=={'INPUT_0'}]
dfRules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(NTZK_TRNSPTMTHDFORSRCHCODE_O),(INPUT_0),0.995121,0.995221,0.990342,0.995197,0.999976,-0.000023,0.995121
7,(NTZK_DCLRTNKINDCODE_E15),(INPUT_0),0.542069,0.995221,0.540675,0.997428,1.002218,0.001197,1.858523
8,(NTZK_UNLOADINGPORTCODE_VNZZZ),(INPUT_0),0.503137,0.995221,0.498855,0.991490,0.996252,-0.001877,0.561641
10,(NTZK_PYMTTERM_KC),(INPUT_0),0.758040,0.995221,0.755850,0.997110,1.001899,0.001432,1.653906
11,(NTZR_HSCODE_2_39),(INPUT_0),0.231803,0.995221,0.231305,0.997852,1.002644,0.000610,2.225311
12,(NTZR_HSCODE_2_48),(INPUT_0),0.201932,0.995221,0.201932,1.000000,1.004802,0.000965,inf
18,"(NTZK_TRNSPTMTHDFORSRCHCODE_O, NTZK_DCLRTNKIND...",(INPUT_0),0.537190,0.995221,0.535796,0.997405,1.002195,0.001173,1.841795
22,"(NTZK_TRNSPTMTHDFORSRCHCODE_O, NTZK_UNLOADINGP...",(INPUT_0),0.503137,0.995221,0.498855,0.991490,0.996252,-0.001877,0.561641
25,"(NTZK_PYMTTERM_KC, NTZK_TRNSPTMTHDFORSRCHCODE_O)",(INPUT_0),0.753161,0.995221,0.750971,0.997091,1.001880,0.001409,1.643261
28,"(NTZK_TRNSPTMTHDFORSRCHCODE_O, NTZR_HSCODE_2_39)",(INPUT_0),0.230509,0.995221,0.230011,0.997840,1.002632,0.000604,2.212885


# mlextend

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [6]:
apriori(df)

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


# apyori

In [116]:
from apyori import apriori
rule = list(apriori(dataset))

In [129]:
rule[-1][2]

[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Kidney Beans', 'Milk', 'Onion', 'Yogurt', 'Eggs', 'Nutmeg'}), confidence=0.2, lift=1.0),
 OrderedStatistic(items_base=frozenset({'Eggs'}), items_add=frozenset({'Onion', 'Yogurt', 'Nutmeg', 'Milk', 'Kidney Beans'}), confidence=0.25, lift=1.25),
 OrderedStatistic(items_base=frozenset({'Kidney Beans'}), items_add=frozenset({'Onion', 'Yogurt', 'Eggs', 'Nutmeg', 'Milk'}), confidence=0.2, lift=1.0),
 OrderedStatistic(items_base=frozenset({'Milk'}), items_add=frozenset({'Onion', 'Yogurt', 'Eggs', 'Nutmeg', 'Kidney Beans'}), confidence=0.33333333333333337, lift=0.8333333333333334),
 OrderedStatistic(items_base=frozenset({'Nutmeg'}), items_add=frozenset({'Onion', 'Yogurt', 'Eggs', 'Milk', 'Kidney Beans'}), confidence=0.5, lift=2.5),
 OrderedStatistic(items_base=frozenset({'Onion'}), items_add=frozenset({'Kidney Beans', 'Yogurt', 'Eggs', 'Nutmeg', 'Milk'}), confidence=0.33333333333333337, lift=1.6666666666666667),
 OrderedStatistic(

In [140]:
dfOut = pd.DataFrame(output)
dfOut = dfOut.explode('ordered_statistics')
dfOut[['antecedent', 'consequent', 'confidence', 'lift']] = dfOut.ordered_statistics.apply(pd.Series)
for i in ['items', 'antecedent', 'consequent']:
    dfOut[i] = dfOut[i].str.join(',')
dfOut = dfOut.drop(columns=['ordered_statistics'])
dfOut

,items,support,antecedent,consequent,confidence,lift
0,Eggs,0.8,,Eggs,0.80,1.00
1,Kidney Beans,1.0,,Kidney Beans,1.00,1.00
2,Milk,0.6,,Milk,0.60,1.00
3,Onion,0.6,,Onion,0.60,1.00
4,Yogurt,0.6,,Yogurt,0.60,1.00
5,"Eggs,Kidney Beans",0.8,,"Eggs,Kidney Beans",0.80,1.00
5,"Eggs,Kidney Beans",0.8,Eggs,Kidney Beans,1.00,1.00
5,"Eggs,Kidney Beans",0.8,Kidney Beans,Eggs,0.80,1.00
6,"Eggs,Onion",0.6,,"Eggs,Onion",0.60,1.00
6,"Eggs,Onion",0.6,Eggs,Onion,0.75,1.25


In [148]:
dfOut.query("antecedent != '' & consequent == 'Milk'")

,items,support,antecedent,consequent,confidence,lift
7,"Milk,Kidney Beans",0.6,Kidney Beans,Milk,0.6,1.0


# other

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/BejaminNaibei/dataset/main/Market_Basket_Optimisation.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
np.set_printoptions(precision=4, suppress=True)

In [20]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=7)

In [21]:
model = DecisionTreeClassifier()
model = model.fit(XTrain, yTrain)

In [27]:
model.decision_path(XTrain).toarray()

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [28]:
20/14000

0.0014285714285714286